## Yolov3 tutorial to train custom images on Google Colab

For this tutorial all the dataset are already on the format required in Yolov3 (.txt). We only took all database and upload to the cloud. First with download darknet repository, install all path and build it on the cloud. Later, we modify or network according to the classes that we want it to identify and finally we run our training. Finally we use the calculated weights in order to test it with some images from our test data. 


Please make sure that we select GPU model  (edit/notebook setting/ hardware GPU)


## Step 1: Download darknet and build.

In [0]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [0]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [0]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [0]:
# make darknet (build)
!make

In [0]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

## Step 2: Upload dataset on darknet
First we mount my drive in order to upload our dataset and files to the cloud 

In [0]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s "/content/gdrive/My Drive" "/mydrive"

!ls /mydrive

With this symbolic link we can drag from mydrive (which is our google drive) to darknet folder. The first thing is to drag/copy /obj (this is the folder with images and .txt data) to darknet/data. Secondly, drag obj.data and obj.names to /darknet/data. It is important that our obj.data have the following path:

classes = 1
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup=/mydrive/yolov3/backup

In obj.names please type the classes names (i.e brick, person, traffic_light,etc)

For cfg file, create a copy a rename as yolov3-custom.cfg and modify max_batches=(#clasess x 2000) 
filters =(#classes+5)*3 
steps=80%max_batches, 90%*match_batches
Use  CTRL+F type yolo and modify classes=#classes (line 696) and in the three sections of yolo [convolutional] modify filters according to the eq. (#clases+5)*3. Save on my drive a drag to /darknet/cfg on the cloud. 

The last thing to is create manually a folder namely /backup on mydrive


## STEP 3: training 
Into /darknet clone and download darnket53.conv.74 

In [0]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit Enter

>function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)


Now, we're ready to train

In [0]:
# train your custom detector
!./darknet detector train data/obj.data cfg/yolov3-custom.cfg darknet53.conv.74 -dont_show

The training will save every 100 iterations a weight file into our backup folder (this is why we create it). Also we can see the chart with loss in a chart.png file 

In [0]:
imShow('chart_yolov3-custom.png')

 If for some reason you get an error or your Colab goes idle during training, you have not lost your weights! Every 100 iterations a weights file called yolov3_custom_last.weights is saved to mydrive/yolov3/backup/ folder (wherever your backup folder is). This is why we created this folder in our Google drive and not on the cloud VM. If your runtime crashes and your backup folder was in your cloud VM you would lose your weights and your training progress.

We can kick off training from our last saved weights file so that we don't have to restart. Just run the following command but with your backup location.

In [0]:
!./darknet detector train data/obj.data cfg/yolov3-custom.cfg /mydrive/yolov3/backup/yolov3-custom_last.weights -dont_show

## Step 4: Run the custom object detector

In [0]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..

In [0]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov3-custom.cfg /mydrive/yolov3/backup/yolov3-custom_last.weights /mydrive/images/**.jpg -thresh 0.3
imShow('predictions.jpg')

## DONE. Have Fun 

Special thanks to @The AI guy from his youtube channel. 